In [2]:
import os, re
from collections import defaultdict as ddict, Counter

import numpy as np
import pandas as pd
from matplotlib.figure import Figure
from gzip import open as gzopen

In [37]:
%matplotlib inline

In [3]:
f = "../misc_data/1KGQ_common_pop_phased.vcf.gz"

In [4]:
contigs = []
skipi = 0
with gzopen(f, 'rt') as inp:
    for line in inp:
      skipi += 1
      if line.startswith('##contig'):
        m = re.search('ID=([^,>]+),?', line)
        if not m:
           print(line)
           break
        contigs.append(m[1])
      if line.startswith('##'): continue
      fields = line.strip().split()
      samples = fields[9:]
      break

In [ ]:
', '.join(contigs)

'chr1, chr2, chr3, chr4, chr5, chr6, chr7, chr8, chr9, chr10, chr11, chr12, chr13, chr14, chr15, chr16, chr17, chr18, chr19, chr20, chr21, chr22'

In [8]:
def process_line(line):
    data = line.strip().split()
    xrm = data[0]
    pos = int(data[1])
    ref = data[3]
    alts = data[4].split(',')
    alt_types = [1 if len(ref) != len(a) else 0 for a in alts]

    info = { k: v for k, v in [d.split('=') for d in data[7].split(';')] }
    form = data[8].split(':')
    gti = form.index('GT')

    counts = [0, 0]
    for sdat in data[9:]:
        gt = sdat.split(':')[gti]
        sals = gt[0], gt[2]
        for s in sals:
            if s not in '.0':
                t = alt_types[int(s)-1]
                counts[t] += 1
    
    return xrm, pos, counts


In [10]:
vdata = ddict(list)
form = "\r{} {}"
with gzopen(f, 'rt') as inp:
    for i in range(skipi):
        inp.readline()

    for line in inp:
        xrm, pos, (snps, indels) = process_line(line)
        print(form.format(xrm, pos), end='')
        if xrm != 'chr1': break
        vdata['xrm'].append(xrm)
        vdata['pos'].append(pos)
        vdata['nsnps'].append(snps)
        vdata['ninds'].append(indels)
        

chr2 100726405

In [11]:
df = pd.DataFrame(vdata)

In [16]:
df.set_index('pos', inplace=True)

In [19]:
df.loc[4e6 : 9e6]

,xrm,nsnps,ninds
pos,,,
4000027,chr1,2,0
4000054,chr1,7,0
4000056,chr1,13,0
4000062,chr1,2,0
4000090,chr1,2,0
...,...,...,...
8999730,chr1,4,0
8999779,chr1,600,0
8999913,chr1,2,0


In [1]:
fig = Figure(figsize=(10,3), layout=None)
ax = fig.subplots()

fig.suptitle('X')

bins = 30

L = df.index[-1]

B = np.linspace(0, L, bins+1)
X = np.histogram(df.nsnps, bins=B)
ax.bar(L[:-1], X, L/bins)

#fig.set_size()
fig

NameError: name 'Figure' is not defined

In [33]:
fig.subplotpars.left

0.125